In [1]:
import sys
sys.path.insert(0, "/home/uih/JYL/Programs/YOLO/")
import torch
from models import Yolov5Small

In [2]:
class_num = 80

In [3]:
yolo = Yolov5Small(3, class_num)
my_state_dict = yolo.state_dict()

In [5]:
my_total_k = []
my_total_v = []
my_focus_layer = []
my_stage_1_conv = []
for i, (k, v) in enumerate(my_state_dict.items()):
    print(f'{i}: {k}\t\t{v.shape}')
    my_total_k.append(k)
    my_total_v.append(v)
    if "focus" in k:
        my_focus_layer.append(v)
        continue
    if "stage1_conv" in k:
        my_stage_1_conv.append(v)

0: focus.conv.weight		torch.Size([32, 3, 6, 6])
1: focus.bn.weight		torch.Size([32])
2: focus.bn.bias		torch.Size([32])
3: focus.bn.running_mean		torch.Size([32])
4: focus.bn.running_var		torch.Size([32])
5: focus.bn.num_batches_tracked		torch.Size([])
6: backbone_stage1_conv.conv.weight		torch.Size([64, 32, 3, 3])
7: backbone_stage1_conv.bn.weight		torch.Size([64])
8: backbone_stage1_conv.bn.bias		torch.Size([64])
9: backbone_stage1_conv.bn.running_mean		torch.Size([64])
10: backbone_stage1_conv.bn.running_var		torch.Size([64])
11: backbone_stage1_conv.bn.num_batches_tracked		torch.Size([])
12: backbone_stage1_bscp.cba1.conv.weight		torch.Size([32, 64, 1, 1])
13: backbone_stage1_bscp.cba1.bn.weight		torch.Size([32])
14: backbone_stage1_bscp.cba1.bn.bias		torch.Size([32])
15: backbone_stage1_bscp.cba1.bn.running_mean		torch.Size([32])
16: backbone_stage1_bscp.cba1.bn.running_var		torch.Size([32])
17: backbone_stage1_bscp.cba1.bn.num_batches_tracked		torch.Size([])
18: backbone_stage1_b

In [6]:
v5_state_dict = torch.load('/home/uih/JYL/Programs/yolov5_version6/torch_state_dict_yolos.pth', map_location='cpu')

In [7]:
v5_total_k, v5_total_v = [], []
v5_focus_layer = []
v5_stage_1_conv = []
for i, (k, v) in enumerate(v5_state_dict.items()):
    if 'anchor' not in k:
        v5_total_k.append(k)
        v5_total_v.append(v)
        print(f'{i}: {k}\t\t{v.shape}')
    if "model.0" in k:
        v5_focus_layer.append(v)
    if "model.1" in k:
        v5_stage_1_conv.append(v)

0: model.0.conv.weight		torch.Size([32, 3, 6, 6])
1: model.0.bn.weight		torch.Size([32])
2: model.0.bn.bias		torch.Size([32])
3: model.0.bn.running_mean		torch.Size([32])
4: model.0.bn.running_var		torch.Size([32])
5: model.0.bn.num_batches_tracked		torch.Size([])
6: model.1.conv.weight		torch.Size([64, 32, 3, 3])
7: model.1.bn.weight		torch.Size([64])
8: model.1.bn.bias		torch.Size([64])
9: model.1.bn.running_mean		torch.Size([64])
10: model.1.bn.running_var		torch.Size([64])
11: model.1.bn.num_batches_tracked		torch.Size([])
12: model.2.cv1.conv.weight		torch.Size([32, 64, 1, 1])
13: model.2.cv1.bn.weight		torch.Size([32])
14: model.2.cv1.bn.bias		torch.Size([32])
15: model.2.cv1.bn.running_mean		torch.Size([32])
16: model.2.cv1.bn.running_var		torch.Size([32])
17: model.2.cv1.bn.num_batches_tracked		torch.Size([])
18: model.2.cv2.conv.weight		torch.Size([32, 64, 1, 1])
19: model.2.cv2.bn.weight		torch.Size([32])
20: model.2.cv2.bn.bias		torch.Size([32])
21: model.2.cv2.bn.running_me

In [8]:
len(my_total_k), len(v5_total_k)

(348, 348)

In [9]:
for i in range(len(my_total_k)):
    if my_total_v[i].shape != v5_total_v[i].shape:
        print(i)

## load prarmeters

In [10]:
i = 0
total = v5_total_v[:]
for m in yolo.modules():
    # if i >= 362:
    #     continue
    if isinstance(m, torch.nn.Conv2d):
        m.weight.data = total[i]
        print(m.weight.data.shape, total[i].shape)
        i += 1
        if m.bias is not None:
            m.bias.data = total[i]
            i += 1
        
    if isinstance(m, torch.nn.BatchNorm2d):
        m.weight.data = total[i]
        i += 1
        m.bias.data = total[i]
        i += 1
        m.running_mean = total[i]
        i += 1
        m.running_var = total[i]
        i += 1
        m.num_batches_tracked = total[i]
        i += 1

torch.Size([32, 3, 6, 6]) torch.Size([32, 3, 6, 6])
torch.Size([64, 32, 3, 3]) torch.Size([64, 32, 3, 3])
torch.Size([32, 64, 1, 1]) torch.Size([32, 64, 1, 1])
torch.Size([32, 64, 1, 1]) torch.Size([32, 64, 1, 1])
torch.Size([64, 64, 1, 1]) torch.Size([64, 64, 1, 1])
torch.Size([32, 32, 1, 1]) torch.Size([32, 32, 1, 1])
torch.Size([32, 32, 3, 3]) torch.Size([32, 32, 3, 3])
torch.Size([128, 64, 3, 3]) torch.Size([128, 64, 3, 3])
torch.Size([64, 128, 1, 1]) torch.Size([64, 128, 1, 1])
torch.Size([64, 128, 1, 1]) torch.Size([64, 128, 1, 1])
torch.Size([128, 128, 1, 1]) torch.Size([128, 128, 1, 1])
torch.Size([64, 64, 1, 1]) torch.Size([64, 64, 1, 1])
torch.Size([64, 64, 3, 3]) torch.Size([64, 64, 3, 3])
torch.Size([64, 64, 1, 1]) torch.Size([64, 64, 1, 1])
torch.Size([64, 64, 3, 3]) torch.Size([64, 64, 3, 3])
torch.Size([256, 128, 3, 3]) torch.Size([256, 128, 3, 3])
torch.Size([128, 256, 1, 1]) torch.Size([128, 256, 1, 1])
torch.Size([128, 256, 1, 1]) torch.Size([128, 256, 1, 1])
torch.Si

In [11]:
i

348

In [12]:
i = 0
total = v5_total_v[:]
for m in yolo.modules():
    # if i >= 362:
    #     continue
    if isinstance(m, torch.nn.Conv2d):
        assert (m.weight.data == total[i]).all()
        i += 1
        if m.bias is not None:
            assert (m.bias.data == total[i]).all()
            i += 1
    if isinstance(m, torch.nn.BatchNorm2d):
        assert (m.weight.data == total[i]).all()
        i += 1
        assert (m.bias.data == total[i]).all()
        i += 1
        assert (m.running_mean == total[i]).all()
        i += 1
        assert (m.running_var == total[i]).all()
        i += 1
        assert (m.num_batches_tracked == total[i]).all()
        i += 1

In [13]:
my_output_state = yolo.state_dict()
torch.save({'model_state_dict': my_output_state}, "./yolov5s_for_coco.pth")